In [2]:
# Import libraries
import numpy as np 
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [3]:
# Prepare Dataset
# load data

# relative path to npz files
path = 'Measurements/Sensor-based-dome-2'
file_name = 'output_batch_%d.npz'

# Training
# how many frames to load
frames_num = 5
data_frames = []
data_forces = []
for i in range(frames_num):
    file_path = os.path.join(path,file_name %i)
    data = np.load(file_path)
    data_frames.append(data['frames'])
    data_forces.append(data['forces'])

combined_frames = np.concatenate(data_frames, axis=0)
combined_forces = np.concatenate(data_forces, axis=0)

features_train = combined_frames
targets_train = combined_forces[:,2] # only z axis force

features_train = torch.from_numpy(features_train)
targets_train = torch.from_numpy(targets_train)

# Test
# file_path = os.path.join(path, file_name %11)
# test_data = np.load(file_path)

In [3]:
# train test split. Size of train data is 10/11 and size of test data is 1/11 
# features_train, features_test, targets_train, targets_test = train_test_split(combined_frames,
#                                                                              combined_forces[:,2],
#                                                                              test_size = (1/11),
#                                                                              random_state = 42) 
# train_test_split NOT RECOMMENDED, because it shuffles temporally dependent data

# features_test = test_data['frames']
# targets_test = test_data['forces']
# targets_test = targets_test[:,2]

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable

# create feature and targets tensor for test set. 
# featuresTest = torch.from_numpy(features_test)
# targetsTest = torch.from_numpy(targets_test)

In [4]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_channels, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 9, kernel_size = 5, stride = 1, padding = 2)
        self.pool = nn.MaxPool2d(kernel_size = 4, stride = 4)
        self.conv2 = nn.Conv2d(9, 18, kernel_size = 5, stride = 1, padding = 2) # If needed
        
        # RNN
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        # Input size is the number of convolutional filters of the previous layer times the size
        # of the image after the last pooling
        self.rnn = nn.RNN(18 * 16 * 30, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Convolutional layers
        x = self.pool(nn.ELU()(self.conv1(x)))
        x = self.pool(nn.ELU()(self.conv2(x))) # If needed
        
        # Reshape for RNN
        x = torch.reshape(x, (-1, 1, 18 * 16 * 30))  # Reshape to (batch_size, seq_len, input_size)
        print("Size of x:", x.size())  # Print size of x
        
        # RNN
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        #print("Size of h0:", h0.size())  # Print size of h0 - Debugging - Obsolete
        out, hn = self.rnn(x, h0)
        
        # Output layer
        out = self.fc(out[:, -1, :]) 
        return out


In [5]:
# batch_size, epoch and iteration
batch_size = 1000
num_epochs = 5

# Pytorch train and test sets
train = TensorDataset(featuresTrain,targetsTrain)
# test = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
# test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    
# Create RNN
input_channels = 3  # RGB channels
hidden_dim = 200  # hidden layer dimension
layer_dim = 2     # number of hidden layers
output_dim = 1   # output dimension

model = RNNModel(input_channels, hidden_dim, layer_dim, output_dim)

# Define your loss function
error = nn.MSELoss()

# Define your optimizer
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Training
seq_dim = 50 # Consider the whole batch to be temporally correlated
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    print('~~~BEGINNING OF DATASET~~~')
    for i, (images, labels) in enumerate(train_loader):
        images = images.float()
        # print(images.shape)  # Add this line to check the shape of images - Debugging purposes
        images = images.permute(0, 3, 1, 2)
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(images)
        outputs = torch.squeeze(outputs)
        
        # Calculate loss
        loss = error(outputs, labels.float())
        
        # Backpropagation
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
            
        # Store loss and iteration
        loss_list.append(loss.data)
        # Print Loss
        if count % 1 == 0: # for now print for every iteration
            print('Iteration: {}  Loss: {}'.format(count, loss.data.item()))

~~~BEGINNING OF DATASET~~~
Size of x: torch.Size([1000, 1, 8640])
Iteration: 1  Loss: 1.6021342277526855
Size of x: torch.Size([1000, 1, 8640])


In [ ]:
print(loss_list)
print(np.sqrt(np.mean(loss_list[-10:])))

[tensor(263.5344), tensor(318.0808), tensor(175.7530), tensor(4.9789), tensor(63.0799), tensor(129.6409), tensor(88.4487), tensor(20.5503), tensor(1.6785), tensor(31.3393), tensor(59.6225), tensor(40.5326), tensor(10.9264), tensor(1.3135), tensor(14.9198), tensor(27.9537), tensor(22.2273), tensor(7.6367), tensor(1.1139), tensor(3.9478), tensor(11.4169), tensor(11.4815), tensor(7.5084), tensor(2.8617), tensor(1.1848), tensor(3.3818), tensor(5.5569), tensor(5.5407), tensor(4.1380), tensor(1.6186), tensor(1.1567), tensor(1.7834), tensor(2.5494), tensor(2.5374), tensor(1.9607), tensor(1.4184), tensor(1.1095), tensor(1.3553), tensor(1.5855), tensor(1.3069), tensor(1.0281), tensor(0.9966), tensor(1.1966), tensor(1.1213), tensor(1.3447), tensor(1.4265), tensor(0.9916), tensor(0.9639), tensor(0.9287), tensor(1.0158)]
1.049458


In [ ]:
print(outputs) # Small sanity check

tensor([-1.4390, -1.6798, -1.3761, -1.6742, -1.4726, -1.5396, -1.4566, -1.6832,
        -1.6207, -1.5412, -1.4651, -1.2927, -1.1839, -0.9136, -1.0314, -0.6858,
        -0.8589, -0.7863, -0.9078, -1.0000, -1.3303, -1.1615, -1.5106, -1.2541,
        -1.5544, -1.5094, -1.5063, -1.5673, -1.7674, -1.3458, -1.8002, -1.6865,
        -1.6369, -1.6120, -1.5567, -1.4272, -1.6723, -1.4395, -1.1112, -0.9764,
        -1.0229, -0.8187, -0.7604, -0.7127, -0.9836, -0.9059, -1.1708, -1.1666,
        -1.3527, -1.4642, -1.5702, -1.4378, -1.5149, -1.5482, -1.6335, -1.5511,
        -1.4784, -1.3611, -1.3325, -1.5687, -1.4828, -1.2066, -0.9760, -1.0698,
        -0.9529, -0.6747, -0.6082, -0.7939, -0.7483, -0.8993, -0.8894, -1.0418,
        -0.9757, -1.2215, -1.3234, -1.5811, -1.5214, -1.5055, -1.4919, -1.7612,
        -1.6361, -1.5542, -1.4665, -1.4462, -1.4591, -1.4921, -1.6085, -1.2376,
        -1.0455, -0.8925, -0.7541, -0.5016, -0.5448, -0.6175, -0.8644, -0.5999,
        -0.8163, -0.9700, -1.1658, -1.27

In [ ]:
print(labels.float())

tensor([-1.0797, -0.9949, -0.9238, -0.8581, -0.7988, -0.7402, -0.6864, -0.6413,
        -0.6007, -0.5637, -0.5303, -0.5034, -0.4781, -0.4516, -0.4438, -0.4478,
        -0.4525, -0.4622, -0.4775, -0.4831, -0.4824, -0.4824, -0.4840, -0.4829,
        -0.4796, -0.4784, -0.4745, -0.4723, -0.4696, -0.4686, -0.4738, -0.4907,
        -0.5179, -0.5654, -0.6264, -0.6943, -0.7702, -0.8488, -0.9312, -1.0187,
        -1.1152, -1.2284, -1.3486, -1.4846, -1.6458, -1.8335, -2.0371, -2.2583,
        -2.4803, -2.6908, -2.8747, -3.0401, -3.1835, -3.2991, -3.3773, -3.4140,
        -3.4004, -3.3335, -3.2181, -3.0745, -2.9110, -2.7374, -2.5585, -2.3805,
        -2.1968, -2.0178, -1.8431, -1.6839, -1.5378, -1.3989, -1.2665, -1.1307,
        -0.9906, -0.8688, -0.7793, -0.7269, -0.7026, -0.7065, -0.7217, -0.7390,
        -0.7422, -0.7402, -0.7296, -0.7031, -0.6639, -0.6237, -0.5844, -0.5494,
        -0.5165, -0.4928, -0.4744, -0.4658, -0.4598, -0.4682, -0.4804, -0.4992,
        -0.5257, -0.5651, -0.6205, -0.69

Progress Summary: 2 Conv&Pool layers, RNN with 2 hidden states of 200 size, 1000 sequence length, 5 epochs: 1.05 RMSE. First good performance, albeit on training set.